In [2]:
#@title Get text train and test ,X and Y
nrows_train = 20000 # @param {type:"integer"}
sequence_length = 13 # @param {type:"integer"}

feed_lenght = 8

import pandas as pd
import numpy as np
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import json
from keras.preprocessing.text import tokenizer_from_json

#load fusion
fus = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/gpt/LEST/LESTfusion1.csv",
                  parse_dates=["time"], index_col="time")

#Get the train and test train
texts_train = fus["fusion"].sample(nrows_train,)
texts_test = fus["fusion"].drop(texts_train.index)

#save texts test
texts_test.to_csv("/content/drive/MyDrive/Colab Notebooks/gpt/LEST/LESTtexts_test1.csv")

# Tokenize text
tokenizer = Tokenizer()

#tokenizer.fit_on_texts(texts_train)
tokenizer.fit_on_texts(fus["fusion"])

#Save tokenizer
tokenizer_json = tokenizer.to_json()

# Save the JSON configuration to a file
with open('/content/drive/MyDrive/Colab Notebooks/gpt/LEST/LESTtokenizer1.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

sequences = tokenizer.texts_to_sequences(texts_train)

# Prepare input and output data
X = []
y = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        x_seq = sequence[:i]
        x_seq_padded = pad_sequences([x_seq], maxlen=sequence_length, padding='pre')
        X.append(x_seq_padded[0])
        y.append(sequence[i])

X = np.array(X)
y = np.array(y)

#filter seed/model words
df = pd.DataFrame(X)
df["y"] = y
df_fil =  df[(df.iloc[:, :sequence_length-feed_lenght+1] != 0).any(axis=1)]
X = df_fil.iloc[:, :-1].values
y = df_fil.iloc[:, -1].values

# One hot encode the outputs
y = np.eye(len(tokenizer.word_index) + 1)[y]

print("Dictionary size (words): len(y[i]):",len(y[1]) )
print("Total X variables:",len(X) )

print("\nText train example1")
display(texts_train[0])

print("\nSequences example 1")
display(sequences[0])

print("\nText train example2")
display(texts_train[1])

print("\nSequences example 2")
display(sequences[1])

print("\nX")
display(X[:25])

print("\nDictionary first 10 words")
display({k: tokenizer.word_index[k] for k in list(tokenizer.word_index)[:10]})


Dictionary size (words): len(y[i]): 3115
Total X variables: 181924

Text train example1


'20009kt 9999 prec0n CL5 CM0 10 07 q1026 17005KT 140V210 9999 FEW020 BKN028 09/06 Q1026 NOSIG'


Sequences example 1


[250, 1, 2, 33, 3, 8, 16, 52, 431, 818, 1, 83, 252, 10, 18, 52, 4]


Text train example2


...

[520, 1, 2, 5, 3, 6, 13, 37, 872, 15, 7, 10, 37, 4]


X


array([[  0,   0,   0,   0,   0, 250,   1,   2,  33,   3,   8,  16,  52],
       [  0,   0,   0,   0, 250,   1,   2,  33,   3,   8,  16,  52, 431],
       [  0,   0,   0, 250,   1,   2,  33,   3,   8,  16,  52, 431, 818],
       [  0,   0, 250,   1,   2,  33,   3,   8,  16,  52, 431, 818,   1],
       [  0, 250,   1,   2,  33,   3,   8,  16,  52, 431, 818,   1,  83],
       [250,   1,   2,  33,   3,   8,  16,  52, 431, 818,   1,  83, 252],
       [  1,   2,  33,   3,   8,  16,  52, 431, 818,   1,  83, 252,  10],
       [  2,  33,   3,   8,  16,  52, 431, 818,   1,  83, 252,  10,  18],
       [ 33,   3,   8,  16,  52, 431, 818,   1,  83, 252,  10,  18,  52],
       [  0,   0,   0,   0,   0, 520,   1,   2,   5,   3,   6,  13,  37],
       [  0,   0,   0,   0, 520,   1,   2,   5,   3,   6,  13,  37, 872],
       [  0,   0,   0, 520,   1,   2,   5,   3,   6,  13,  37, 872,  15],
       [  0,   0, 520,   1,   2,   5,   3,   6,  13,  37, 872,  15,   7],
       [  0, 520,   1,   2,   5,   3, 


Dictionary first 10 words


{'9999': 1,
 'prec0n': 2,
 'cm0': 3,
 'nosig': 4,
 'cl0': 5,
 '11': 6,
 '12': 7,
 '10': 8,
 '13': 9,
 '09': 10}

In [3]:
#@title Train the model

from keras.layers import Bidirectional

# Build the LSTM model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=20, input_length=sequence_length),
    Bidirectional(LSTM(130)),

    Dense(len(tokenizer.word_index)+1, activation='softmax')
])


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=50,batch_size=512)

#save the model
model.save("/content/drive/MyDrive/Colab Notebooks/gpt/LEST/LESTmodel1.keras")


Epoch 1/50
356/356 [==============================] - 29s 70ms/step - loss: 3.9472 - accuracy: 0.2076
Epoch 2/50
356/356 [==============================] - 25s 69ms/step - loss: 2.8421 - accuracy: 0.2949
Epoch 3/50
356/356 [==============================] - 24s 68ms/step - loss: 2.5798 - accuracy: 0.3248
Epoch 4/50
356/356 [==============================] - 25s 69ms/step - loss: 2.4432 - accuracy: 0.3412
Epoch 5/50
356/356 [==============================] - 25s 69ms/step - loss: 2.3509 - accuracy: 0.3549
Epoch 6/50
356/356 [==============================] - 24s 68ms/step - loss: 2.2841 - accuracy: 0.3650
Epoch 7/50
356/356 [==============================] - 24s 69ms/step - loss: 2.2296 - accuracy: 0.3745
Epoch 8/50
356/356 [==============================] - 24s 69ms/step - loss: 2.1801 - accuracy: 0.3827
Epoch 9/50
356/356 [==============================] - 25s 69ms/step - loss: 2.1361 - accuracy: 0.3903
Epoch 10/50
356/356 [==============================] - 24s 68ms/step - loss: 2.093